# Location-Based Consumer Preference Inference

Notebook used for inspection of the GeoLife dataset, retrieved from  
https://www.microsoft.com/en-us/research/project/geolife-building-social-networks-using-human-location-history/#!downloads  
  
The description of the data can be found in the path `<PATH TO GeoLife>`/Geolife Trajectories 1.3/User Guide-1.3.pdf  

In [1]:
from math import sqrt, cos, sin, pi
import os.path
import sqlite3
#import collection # c = collection.Counter([lista])
from datetime import datetime
from geopy.distance import lonlat, distance, vincenty
import pandas as pd
import folium
from folium import plugins

In [2]:
#### Retrieve data, based on SELECTION from (SQLite) DB stored in DBFILE, located in PATH
def sqlite_2_pd_df(selection, path, dbfile, db):    
    os.chdir(path)
    con=sqlite3.connect(dbfile)
    con.text_factory = sqlite3.OptimizedUnicode
    query = con.execute("SELECT " + selection + " From " + db)
    cols = [column[0] for column in query.description]
    df= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
    return df

In [3]:
#### Test example
#### The file GeoLifeDBMINI.db holds a SQLite DB with the 
workdir = '/home/patrickm/projects/GeoLife/'
DBfile  = 'GeoLifeDBMINI.db'

#### Function calculating length of a trajectory
def tracklengthGEO(lat, long, alt):
    ln = len(lat)
    #trkseg = [ distance( (lat[i], long[i] ), (lat[i-1], long[i-i] )).m for i in range(ln) ]
    trkseg = [ distance((lat[i], long[i], alt[i]), (lat[i-1], long[i-i], alt[i-1] )).m for i in range(ln) ]
    #trkseg = [ distance.distance((lat[i], long[i], alt[i]), (lat[i-1], long[i-1], alt[i-1])).miles for i in range(ln) ]
    return sum(trkseg)

In [4]:
#### Create dataframe from SQLite DB
df = sqlite_2_pd_df("*", workdir, DBfile, "GeoLifeDBMINI")

### Quick extraction of essential user data. Many thanks to Marcin

user_track = {}
for i, row in enumerate(df.iterrows()):
    row = row[1] # access the relevant part of the data structure
    key = row['usrID'], row['trkID']
    clock_string = row['date str'] + ' ' + row['time str']  # 2008-11-23 10:21:53 formatting
    clock = datetime.strptime(clock_string, '%Y-%m-%d %H:%M:%S')
    
    if key not in user_track: # start iteration over new user
        user_track[key] = {'min': clock, 'max': clock, 'markers': 0}
        
    user_track[key]['markers'] += 1
    if clock > user_track[key]['max']:
        user_track[key]['max'] = clock
    if clock < user_track[key]['min']:
        user_track[key]['min'] = clock

for key in user_track:
    user_track[key]['time between markers'] = (user_track[key]['max'] - user_track[key]['min']).total_seconds()/user_track[key]['markers']

user_track

{(0, '20081123102153'): {'min': datetime.datetime(2008, 11, 23, 10, 21, 53),
  'max': datetime.datetime(2008, 11, 23, 10, 30, 8),
  'markers': 21,
  'time between markers': 23.571428571428573},
 (0, '20090620024527'): {'min': datetime.datetime(2009, 6, 20, 2, 45, 27),
  'max': datetime.datetime(2009, 6, 20, 14, 38, 51),
  'markers': 682,
  'time between markers': 62.76246334310851},
 (0, '20090511174427'): {'min': datetime.datetime(2009, 5, 11, 17, 44, 27),
  'max': datetime.datetime(2009, 5, 11, 18, 0, 48),
  'markers': 201,
  'time between markers': 4.880597014925373},
 (0, '20090527080105'): {'min': datetime.datetime(2009, 5, 27, 8, 1, 5),
  'max': datetime.datetime(2009, 5, 27, 12, 55, 34),
  'markers': 1773,
  'time between markers': 9.965595036661027},
 (0, '20081201111827'): {'min': datetime.datetime(2008, 12, 1, 11, 18, 27),
  'max': datetime.datetime(2008, 12, 1, 11, 35, 15),
  'markers': 183,
  'time between markers': 5.508196721311475},
 (0, '20090611221204'): {'min': dateti

In [5]:
# Pandas gymnastics

tmp = df.copy(deep=True)
tmp.reset_index()
tmp.set_index(['usrID','trkID'], inplace = True)
tmp = tmp.groupby(['usrID', 'trkID']).size().to_frame('no of markers')
print(tmp.head(100))
print(tmp.reset_index())


tmp2 = df.copy(deep=True)
tmp2.reset_index()
#tmp2.set_index(['usrID','trkID'], inplace = True)
print (tmp2.loc[tmp2['usrID']==2].head())

### Unique usrID list
#usrlst = df['usrID'].unique().tolist()
#usrlst.sort()
#print (usrlst)

#usrdct = {}

#for usr in usrlst:
#    print("User : ", usr)
#    tmp = df.copy(deep=True)
#    tmp = tmp[tmp['usrID'] == usr ].sort_values(by='trkID', ascending=True)
#    print(df.head())
#    trklst = tmp['trkID'].unique().tolist()
#    trklst.sort()
#    print(trklst)
#    dct = { trk : trklst.index(trk) for trk in trklst }
#    print (trklst)
#    print(dct)
#    usrdct.update( {usr : dct} )
#    tmp['trk no'] = tmp['trkID'].map(dct)
    #print("# trajectories : ", len())
#df = df.set_index(['usrID'])
#print(df)

#df = df.set_index(['usrID'])
  


                      no of markers
usrID trkID                        
0     20081023025304            908
      20081024020959            244
      20081026134407            745
      20081027115449             50
      20081028003826           1477
      20081029092138             21
      20081029093038            182
      20081103101336              7
      20081103232153           2231
      20081110013637            272
      20081111001704           1477
      20081112023003            681
      20081112091400             27
      20081113034608            494
      20081114015255            337
      20081114101436            305
      20081115010133            281
      20081116085532             12
      20081117051133             81
      20081117155223            314
      20081118095005             32
      20081118095400            254
      20081118162008              7
      20081119012806             81
      20081119112035            419
      20081120035736        

In [18]:
usr = 0

# tracks
mrks = df.loc[df['usrID'] == usr].sort_values(by=['trkID'])
print(mrks)
#print (mrks[['Lat', 'Long', 'Alt']])

#mrks.groupby('trkID')[['Lat','Long']].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()
#print(mrks)
#tmpdct = {}
#for i in mrks['trkID'].unique():
#print(mrks.loc[mrks['trkID']=='20081029092138'])
lat = [lt for lt in mrks.loc[mrks['trkID']==str(20081029092138)]['Lat'] ]
print (lat)
#    tmpdct[i] = {{mrks['Lat'][j]: df['value2'][j]} for j in df[df['name']==i].index}

#print(df.index.name)

#df = df.set_index(['usrID'])
#print(df)
#tracks = df.groupby('trkID')
#print(df)
#for track in
#df.reset_index()

#dft = df.groupby('usrID')#.groupby('trkID')
#dft2= dft.groupby('trkID')

#df.groupby(['usrID','trkID']).size().reset_index().groupby('trkID')[[0]].max()

#print(usrdct[0])

for usr in usrlst:
    tmpdist = []
    tmp = df.copy(deep=True)
    tmp = tmp[tmp['usrID'] == usr]
    #print("usr no: ", usr)
    #print(tmp)#
    for trk, idx in usrdct[usr].items():
        #print("trk, idx : ", trk, idx)
        main = tmp[tmp['trk no'] == idx ]
        #print(main)
        lat = main['Lat'].tolist()
        long = main['Long'].tolist()
        alt = main['Alt'].tolist()
        #print("lat  = ", lat)
        #print("long = ", long)
        #print("alt  = ", alt)
        tmpdist.append(tracklengthGEO(lat,long,alt))
    print("usr, travel = ", usr, sum(tmpdist))    
#tmp = df.loc[0]
#print(tmp)
#main = tmp[tmp['trk no']==351]

#print("dist = ", tracklengthGEO(lat, long, alt))

             Lat        Long  Alt       no days    date str  time str  usrID  \
68106  40.007411  116.319124  112  39744.427211  2008-10-23  10:15:11      0   
67956  40.009247  116.321721  139  39744.413322  2008-10-23  09:55:11      0   
67957  40.009273  116.321781  145  39744.413380  2008-10-23  09:55:16      0   
67958  40.009313  116.321871  154  39744.413437  2008-10-23  09:55:21      0   
67959  40.009344  116.321952  159  39744.413495  2008-10-23  09:55:26      0   
67960  40.009356  116.322007  164  39744.413553  2008-10-23  09:55:31      0   
67961  40.009386  116.322103  163  39744.413611  2008-10-23  09:55:36      0   
67962  40.009356  116.322121  165  39744.413669  2008-10-23  09:55:41      0   
67963  40.009353  116.322135  165  39744.413727  2008-10-23  09:55:46      0   
67964  40.009319  116.322152  165  39744.413785  2008-10-23  09:55:51      0   
67965  40.009394  116.322162  168  39744.413843  2008-10-23  09:55:56      0   
67966  40.009399  116.322179  168  39744

NameError: name 'usrlst' is not defined

In [ ]:
#tmp = df.copy(deep=False)
#main = tmp[tmp['trk no'] == 0 ]
#print("trk = 0")
#print(main.head())
#
#tmp = df.copy(deep=False)
#main = tmp[tmp['trk no'] == 2 ]
#print("trk = 2")
#print(main.head())
#
#tmp = df.copy(deep=False)
#main = tmp[tmp['trk no'] == 4 ]
#print("trk = 4")
#print(main.head())


BEIJING_COORDINATES = ( 39.9062170, 116.3912757 )
# for speed purposes
MAX_RECORDS = 1000
  
# create empty map zoomed in on Beijing
map_beijing = folium.Map(location=BEIJING_COORDINATES, zoom_start=12)
map_beijing.position
# add a marker for every record in the filtered data, use a clustered view

usr = 0

tmp  = df.copy(deep=True)
main = tmp[tmp['trk no'] == 10]
lat  = main['Lat'].tolist()
long = main['Long'].tolist()

coord = []

for lt in lat:
    coord.append( (lt, long[ lat.index(lt) ] ) ) 
    print("coord = ", lt, long[ lat.index(lt) ])
    folium.Marker(location = (lt, long[ lat.index(lt) ] ) ).add_to(map_beijing)
    
#for each in crimedata[0:MAX_RECORDS].iterrows():
#    folium.Marker(location = [each[1]['Latitude'],each[1]['Longitude']]).add_to(mapsf)
#map.save(outfile = "test.html")
map_beijing
